In [1]:
# How changing the dataset sample size effects the calibration methods
# Fix training dataset size and change the calib set samples - best method is one that gets max calib with least data

In [2]:
# imports
import sys
import pandas as pd
sys.path.append('../../') # to access the files in higher directories
sys.path.append('../') # to access the files in higher directories
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import Data.data_provider as dp
import core as cal
from estimators.IR_RF_estimator import IR_RF

In [3]:
# params
seed = 0
runs = 100
calib_methods = ["RF", "Platt" , "ISO", "Rank", "CRF", "VA", "Beta", "Elkan", "tlr"]
metrics = ["acc", "auc", "brier", "ece"]
# data_list = ["spambase", "climate", "QSAR", "bank", "climate", "parkinsons", "vertebral", "ionosphere", "diabetes", "breast", "blod"]
data_list = ["spambase", "climate"]
# data_list = ["synthetic1"] # , "synthetic2"

params = {
    "n_estimators": 100,
    "oob": False,
}

In [4]:
results_dict = {}
for data_name in data_list:
    # Data
    X, y = dp.load_data(data_name, "../../")
    # X, y, tp = dp.make_classification_gaussian_with_true_prob(30, 4, 0)
    # split to train calib test

    data_dict = {}
    for seed in range(runs):
        data = cal.split_train_calib_test(data_name, X, y, 0.3, 0.5, seed)
        # train model
        irrf = IR_RF(n_estimators=params["n_estimators"], oob_score=params["oob"], random_state=seed)
        irrf.fit(data["x_train"], data["y_train"])

        # calibration
        res = cal.calibration(irrf, data, calib_methods, metrics)
        data_dict = cal.update_runs(data_dict, res)
    results_dict.update(data_dict)

table = cal.mean_and_ranking_table(results_dict, metrics, calib_methods, data_list)

--------------------------------- acc
                RF     Platt       ISO      Rank       CRF        VA      Beta     Elkan       tlr
Data                                                                                              
spambase  0.950760  0.950760  0.951484  0.947864  0.950760  0.409848  0.950760  0.950760  0.943519
climate   0.918025  0.926975  0.924321  0.873580  0.918025  0.917099  0.926235  0.918025  0.917099
Mean      0.934393  0.938868  0.937903  0.910722  0.934393  0.663473  0.938497  0.934393  0.930309
Rank      4.500000  2.500000  2.000000  8.000000  4.500000  8.250000  3.000000  4.500000  7.750000
--------------------------------- auc
                RF     Platt       ISO      Rank       CRF        VA      Beta     Elkan       tlr
Data                                                                                              
spambase  0.984895  0.984895  0.984212  0.983106  0.984895  0.984802  0.984895  0.984895  0.979499
climate   0.864999  0.864999  0.8